In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
key = os.getenv('GOOGLE_API_KEY')

In [2]:
import requests

url = 'https://www.googleapis.com/youtube/v3/commentThreads'
video_id = 'NDsO1LT_0lw'
order = 'relevance'
params = {
    'key': key,
    'part': 'snippet,replies',
    'videoId': video_id,
    'max_Results': 100,
    'order': order,
}

response = requests.get(url, params=params)
print(response.status_code)

200


In [3]:
def print_comments(response):
    data = response.json()
    for item in data['items']:
        top_comment = item['snippet']['topLevelComment']['snippet']
        snippet_id = item['id']
        author = top_comment['authorDisplayName']
        text = top_comment['textDisplay']
        print(f'{author} [{snippet_id}]: \n{text}\n')

In [4]:
print_comments(response)

@MrBeast [UgzgrqctaRkXF0Ioydx4AaABAg]: 
BEAST GAMES FINALE DROPS FEBRUARY 13TH! GO WATCH THE REST NOW! <a href="https://unfur.ly/BeastGames">https://unfur.ly/BeastGames</a>

@marwanrehan [Ugx98WuRmVg8lEshD9l4AaABAg]: 
As an Egyptian, this is the first time I see and hear about the things shown in this video! Honestly, this is one of the best videos you&#39;ve ever made. ❤❤

@YouTube [Ugy9mNx9AEZtvO_rplh4AaABAg]: 
تجربة أسطورية 🤯 مرحبًا بك في مصر! 🇪🇬

@oShven [UgxVm6v87tAKMqFMTUh4AaABAg]: 
Doing what most cant and documenting it for the whole world to see. I LOVE IT <a href="UCkszU2WH9gy1mb0dV-11UJg/ePgfY-K2Kp6Mr8oP1oqAwAc"></a>

@georgehorn4718 [Ugwub4CONf0LGIGywbF4AaABAg]: 
We need an extended cut of this, must be hours worth of cool content

@majlatv [UgyCUNCGaC8G2kIGgeR4AaABAg]: 
هذه الحلقة هي الأقوى والأعظم بلا شك! فأنت في مصر… حيث وُلدت الحضارة قبل أن يُكتب التاريخ!

@Hosamart [UgybJ2GWiQybtH80Kyp4AaABAg]: 
بجد فخور جدااا بحضارتنا وزعلان علي تقدير واستغلال غيرنا حاجات موجوده قدامن

In [5]:
# preprocessing challenges:
# - must translate non-english comments to english
# - must remove stopwords and emjois?

In [6]:
def get_replies(comment_id):
    url = 'https://www.googleapis.com/youtube/v3/comments'
    params = {
        'key': key,
        'part': 'snippet',
        'parentId': comment_id,
        'maxResults': 100
    }

    replies = []
    while 1:
        response = requests.get(url, params=params)
        data = response.json()
        replies.extend(data.get('items', []))
        
        if 'nextPageToken' in data:
            params['pageToken'] = data['nextPageToken']
        else:
            break

    # Sort replies by likeCount in descending order
    sorted_replies = sorted(replies, key=lambda x: x['snippet']['likeCount'], reverse=True)
    return sorted_replies

In [7]:
response = get_replies('UgzgrqctaRkXF0Ioydx4AaABAg')

In [8]:
for item in response[:30]:
    user_id = item['snippet']['authorDisplayName']
    text = item['snippet']['textOriginal']
    likes = item['snippet']['likeCount']

    print(f'{user_id} L:{likes}: {text}')

@SirBhogerPlus L:605: Already plan to!
@ayhamyt6143 L:352: LOL
@Son_Harold L:276: Yeah same
I love beast games ❤
@Revanftbl L:143: Ok
@Edik_Vdle L:129: Я легенда
@ayhamyt6143 L:117: ❤
@Aabdul-RehmanN L:100: ✍️🔥
@BramVanGroezen L:98: Okay mrbeast
@90bands-gambling L:73: Yass
@FlexingBot L:50: ayeeeee
@abdulkadrelmuhtedy313 L:49: عطينا شي ايفون او 10الف دولار❤
@Vortex-k7c L:47: Cool
@MhmdMhmd-lg6wo L:47: Mrbeast🎉🎉 your the beast🎉
@Zenariks L:40: Привет зверь
@THEMURATADAMDIR L:37: efsanevi
@ALORMESSAGE L:36: 🎉🎉
@URalivai L:35: I am in 🇧🇩🇧🇩🇧🇩
@Jay.Shri.ram. L:32: 🎉❤
@raphaelaginam5264 L:27: Sup beast
@NohchiYT444 L:26: 3
@Abdurahmon-King L:23: 🎉
@pkrpro L:23: Love from Pakistan
@ayhamyt6143 L:23: 🐐
@Spiderman27360 L:22: How did bro manage to pull this off…
@ReaalFuturePulse L:21: YEss
@ptsplzfriendme1210 L:20: W
@Monkey-SG L:18: Ok
@realstonksofficial L:16: hello :D
@Jay.Shri.ram. L:15: 🚩🚩🚩
@arnavs7578 L:15: Love you
